# ***Coleta final dos dados***
---

## Descrição

A partir dos scripts de teste individuais, criaremos funções para coletar todos os dados e, posteriormente, juntarmos todos os dados em um dataframe final: `now_data_raw.csv` e `.xlsx`.

# Bibliotecas
---

In [1]:
import pandas as pd
import numpy as np
import datetime as dt
from functools import reduce

import sys
sys.path.insert(0, '..')

from bcb import sgs
import ipeadatapy as ip
import pandas_datareader.data as web

# Definindo funções

In [ ]:
'''def get_bcb_data(path, start, end):
    # Lendo a planilha completa
    df1 = pd.read_csv(path)

    # Filtrando apenas para os dados que constam no SGS-BCB
    df2 = df1[df1['where'] == 'BCB']

    # Definindo os nomes e os códigos das séries
    names_df = list(df2['name_df'])
    codes_bcb = list(map(int, df2['code']))

    # Coletando as séries com o API: criando uma lista compacta com os nomes e códigos para coletar todos os dados simultâneamente
    df3 = sgs.get(list(zip(names_df, codes_bcb)), start = start, end = end, multi = True) # multi: coletando diversas séries em uma única requisição
    df3 = df3.reset_index()

    return df3'''

: 

In [2]:
def get_ipea_data(path, year_greater):
    # Lendo a planilha completa
    df1 = pd.read_csv(path)

    # Filtrando apenas para os dados que constam no Ipeadata
    df2 = df1[df1['where'] == 'Ipeadata']

    # Criando lista de códigos
    codes = list(df2['code'])

    # Coletando os dados
    df3 = pd.DataFrame()
    for code in codes:
        df3[code] = ip.timeseries(series = code, yearGreaterThan = year_greater).iloc[:, -1:]
    
    df3 = df3.reset_index()

    return df3

In [3]:
def get_data_yahoo(path, start, end, interval, series):
    # Lendo a planilha completa
    df1 = pd.read_csv(path)

    # Filtrando apenas para os dados que constam no Yahoo! Finance
    df2 = df1[df1['where'] == 'Yahoo! Finance']

    # Criando lista de códigos
    codes = list(df2['code'])

    # Coletando os dados
    df3 = pd.DataFrame()

    for code in codes:
        df3[code] = web.get_data_yahoo(code, start = start, end = end, interval = interval)[series]
    
    df3 = df3.reset_index()

    return df3

# Banco Central do Brasil (BCB)

In [8]:
'''path = 'full_data_dict.csv'
start = '2002-01-01'
end = '2022-09-01'
bcb_raw = get_bcb_data(path, start, end)
bcb_raw'''

bcb_raw = pd.read_csv('../bcb_data_raw.csv')
bcb_raw.drop('Unnamed: 0', axis = 1, inplace = True)
bcb_raw.rename({'ref.date' : 'Date'}, axis = 1, inplace = True)
bcb_raw['Date'] = pd.to_datetime(bcb_raw['Date'])
bcb_raw

,Date,gdp,ibc_br,mon_gdp,ipca,ipca_ex,ipca_exi,ipca_exii,ipca_exiii,inpc,...,household_debt,steel_prod,road_flow_gross_vehi,cni_ind_ent_confindex,serv_conf_index_fgv,serv_curr_situ_index_fgv,serv_expec_index_fgv,ind_conf_cons_feco,ind_eco_cond_feco,ind_fut_expec_feco
0,2002-01-01,110.63,NaN,112374.8,0.52,0.49,0.74,0.44,0.44,1.07,...,NaN,119.87,109.62,59.86,NaN,NaN,NaN,87.04,70.70,97.93
1,2002-02-01,NaN,NaN,111477.1,0.36,0.83,0.73,0.42,0.46,0.31,...,NaN,113.72,111.46,NaN,NaN,NaN,NaN,90.66,74.34,101.54
2,2002-03-01,NaN,NaN,118444.7,0.60,0.43,0.52,0.51,0.54,0.62,...,NaN,121.24,111.49,NaN,NaN,NaN,NaN,91.30,77.69,100.37
3,2002-04-01,115.41,NaN,120385.9,0.80,0.51,0.50,0.50,0.53,0.68,...,NaN,111.49,110.64,58.97,NaN,NaN,NaN,96.00,74.39,110.41
4,2002-05-01,NaN,NaN,123552.5,0.21,0.52,0.47,0.38,0.53,0.09,...,NaN,123.30,110.41,NaN,NaN,NaN,NaN,96.15,74.99,110.25
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
245,2022-06-01,NaN,141.93,813425.8,0.67,0.77,0.90,0.98,1.01,0.62,...,33.52,147.17,163.01,NaN,95.5,94.1,97.2,103.60,66.19,128.53
246,2022-07-01,NaN,148.85,846022.3,-0.68,0.43,0.36,0.67,0.63,-0.60,...,33.64,142.07,163.66,57.80,98.0,97.2,99.1,105.58,70.01,129.29
247,2022-08-01,NaN,149.01,844613.3,-0.36,0.50,0.54,1.08,1.10,-0.31,...,NaN,142.97,163.15,NaN,98.9,97.1,100.6,106.76,70.19,131.14
248,2022-09-01,NaN,NaN,NaN,-0.29,0.26,0.47,0.64,0.69,-0.32,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# BCB Focus - Expectativas de mercado 

In [9]:
focus_raw = pd.read_csv('focus_data.csv')
focus_raw.drop('Unnamed: 0', axis = 1, inplace = True)
focus_raw['Date'] = pd.to_datetime(bcb_raw['Date'])
focus_raw.dtypes

Date             datetime64[ns]
expec_gdp               float64
expec_ipca              float64
expec_selic             float64
expec_exr_usd           float64
dtype: object

# Ipeadata

In [10]:
path = 'full_data_dict.csv'
year_greater = 2001
ipea_raw = get_ipea_data(path, year_greater)
ipea_raw.columns = ['Date', 'exp_fob', 'exp_kgood_fob', 'exp_comgood_fob', 'imp_fob', 'imp_kgood_fob', 'imp_comgood_fob', 'revenue_rf']
ipea_raw

,Date,exp_fob,exp_kgood_fob,exp_comgood_fob,imp_fob,imp_kgood_fob,imp_comgood_fob,revenue_rf
0,2002-01-01,3952.038148,677.815093,824.920064,3886.422498,701.691574,373.487327,22680.845000
1,2002-02-01,3652.691211,383.777102,957.020453,3483.558340,554.428338,341.300112,17503.650936
2,2002-03-01,4251.755321,462.735576,1028.145481,3749.115387,550.711189,442.221529,17285.790167
3,2002-04-01,4612.069084,448.902045,1016.324080,4225.843874,621.889543,424.275893,19831.720577
4,2002-05-01,4424.075300,383.111035,1027.540607,4154.270359,585.755356,395.439583,18065.018868
...,...,...,...,...,...,...,...,...
244,2022-05-01,29693.962162,1265.425961,3596.463541,24693.623429,2304.678716,2217.890152,153448.964730
245,2022-06-01,32794.321939,1412.617798,4000.652803,23861.224808,2139.208442,2317.005824,166705.347128
246,2022-07-01,29877.158955,1179.566945,3850.927242,24496.868872,2191.215511,2133.871057,181804.319492
247,2022-08-01,30714.227340,1482.854880,4288.503054,26687.486882,2654.146007,2696.197518,NaN


# Yahoo! Finance
---

In [11]:
path = 'full_data_dict.csv'
start = '2002-01-01'
end = '2022-09-01'
interval = 'm'
series = 'Adj Close'
yahoo_raw = get_data_yahoo(path, start, end, interval, series)
# yahoo_raw.drop('^N100', axis = 1, inplace = True)
yahoo_raw.columns = ['Date', 'brent_oil_price', 'ibov', 'nasdaq', 'snp_500', 'dji']
yahoo_raw 

,Date,brent_oil_price,ibov,nasdaq,snp_500,dji
0,2007-08-01,72.690002,54637,2596.360107,1473.989990,13357.740234
1,2007-09-01,79.169998,60465,2701.500000,1526.750000,13895.629883
2,2007-10-01,90.629997,65318,2859.120117,1549.380005,13930.009766
3,2007-11-01,88.260002,63006,2660.959961,1481.140015,13371.719727
4,2007-12-01,93.849998,63886,2652.280029,1468.359985,13264.820312
...,...,...,...,...,...,...
151,2022-04-01,109.339996,107876,12334.639648,4131.930176,32977.210938
152,2022-06-01,114.809998,98542,11028.740234,3785.379883,30775.429688
153,2022-07-01,110.010002,103165,12390.690430,4130.290039,32845.128906
154,2022-08-01,96.489998,109523,11816.200195,3955.000000,31510.429688


# Juntando as bases de dados
---

In [12]:
# Lendo base de dados da FGV com os índices de confiança 

fgv_raw = pd.read_excel('indices_conf_fgv.xlsx')
fgv_raw.columns = ['Date', 'emp_conf_index_fgv', 'emp_curr_situ_index_fgv', 'emp_expec_index_fgv', 'ind_conf_index_fgv', 
                   'ind_curr_situ_index_fgv', 'ind_expec_index_fgv', 'cons_conf_index_fgv', 'cons_curr_situ_index_fgv', 
                   'cons_expec_index_fgv']
fgv_raw.set_index(['Date'], inplace = True)
fgv_raw = fgv_raw/1000
fgv_raw = fgv_raw.reset_index()
fgv_raw = fgv_raw[fgv_raw['Date'] >= '2002-01-01'] 
fgv_raw.dtypes

Date                        datetime64[ns]
emp_conf_index_fgv                 float64
emp_curr_situ_index_fgv            float64
emp_expec_index_fgv                float64
ind_conf_index_fgv                 float64
ind_curr_situ_index_fgv            float64
ind_expec_index_fgv                float64
cons_conf_index_fgv                float64
cons_curr_situ_index_fgv           float64
cons_expec_index_fgv               float64
dtype: object

In [13]:
# Lendo base de dados Google Trends

gtrends_data = pd.read_csv('gtrends_data.csv')
gtrends_data.drop('Unnamed: 0', axis = 1, inplace = True)
gtrends_data['Date'] = pd.to_datetime(gtrends_data['Date'])

In [14]:
dfs = [bcb_raw, fgv_raw, focus_raw, ipea_raw, yahoo_raw, gtrends_data]

now_data_raw = reduce(lambda left, right: pd.merge(left, right, on = ['Date'], how = 'outer'), dfs)

# Deslocando duas linhas abaixo para as séries trimestrais para que o lançamento dos dados fique no fim do período

now_data_raw[['gdp', 'nuci_gv', 'cni_ind_ent_confindex']] = now_data_raw[['gdp', 'nuci_gv', 'cni_ind_ent_confindex']].shift(2)

# Escrevendo em csv e xlsx

now_data_raw.to_excel('now_data_raw.xlsx')
now_data_raw.to_csv('now_data_raw.csv')

In [15]:
now_data_raw

,Date,gdp,ibc_br,mon_gdp,ipca,ipca_ex,ipca_exi,ipca_exii,ipca_exiii,inpc,...,dji,gt_crise,gt_desemprego,gt_seguro_desemprego,gt_empregos,gt_trabalho,gt_fgts,gt_renda,gt_salario,gt_economia
0,2002-01-01,NaN,NaN,112374.8,0.52,0.49,0.74,0.44,0.44,1.07,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2002-02-01,NaN,NaN,111477.1,0.36,0.83,0.73,0.42,0.46,0.31,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2002-03-01,110.63,NaN,118444.7,0.60,0.43,0.52,0.51,0.54,0.62,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2002-04-01,NaN,NaN,120385.9,0.80,0.51,0.50,0.50,0.53,0.68,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2002-05-01,NaN,NaN,123552.5,0.21,0.52,0.47,0.38,0.53,0.09,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
245,2022-06-01,175.78,141.93,813425.8,0.67,0.77,0.90,0.98,1.01,0.62,...,30775.429688,21.0,17.0,14.0,17.0,32.0,12.0,16.0,15.0,5.0
246,2022-07-01,NaN,148.85,846022.3,-0.68,0.43,0.36,0.67,0.63,-0.60,...,32845.128906,16.0,17.0,14.0,16.0,26.0,9.0,14.0,15.0,4.0
247,2022-08-01,NaN,149.01,844613.3,-0.36,0.50,0.54,1.08,1.10,-0.31,...,31510.429688,19.0,18.0,14.0,17.0,31.0,9.0,13.0,16.0,6.0
248,2022-09-01,NaN,NaN,NaN,-0.29,0.26,0.47,0.64,0.69,-0.32,...,28725.509766,19.0,17.0,13.0,14.0,31.0,7.0,10.0,15.0,5.0


In [16]:
now_data_raw.columns

Index(['Date', 'gdp', 'ibc_br', 'mon_gdp', 'ipca', 'ipca_ex', 'ipca_exi',
       'ipca_exii', 'ipca_exiii', 'inpc',
       ...
       'dji', 'gt_crise', 'gt_desemprego', 'gt_seguro_desemprego',
       'gt_empregos', 'gt_trabalho', 'gt_fgts', 'gt_renda', 'gt_salario',
       'gt_economia'],
      dtype='object', length=161)

# Observações

1. Completar os NAs da série do salério mínimo com 1212,00 reais pois a base tem apenas esses NAs quando o valor do SM em voga era esse. Provável erro de cômputo na base de dados.

2. Descartar série de horas trabalhadas na produção (hrs_wkd_prod) da fiesp: série incompleta. 

3. Descartar série de Utilização da capacidade instalada - Geral (CNI) (uci_cni): série incompleta.

4. Descartar série de vendas de máquinas agrícolas (agri_mach_sales): série incompleta

5. Para o fluxo de veículos (road_flow_gross_vehi), vamos precisar de apenas uma decomposição para estacionarizar a série, dado que sua série já existe apenas na forma dessazonalizada, poupando uma defasagem.

6. Decobrir coleta de PMIs